In [1]:
import pandas as pd
import numpy as np


Generate Product Reviews with Reviewer Information

In [2]:
dfr = pd.read_csv('Reviewer_Information.csv')

In [3]:
dfr.Recent_ratings = dfr.Recent_ratings.str.split()

In [4]:
dfr.dropna(axis=0,subset=['Recent_ratings'], inplace=True)

In [5]:
#type(dfrt.Recent_ratings.iloc[1][0])

In [6]:
for i in range(len(dfr)):
    dfr.Recent_ratings.iloc[i] = map(int, dfr.Recent_ratings.iloc[i])

In [7]:
dfr['Recent_rating_mean']=None
dfr['Recent_rating_SD'] = None

In [8]:
for i in range(len(dfr)):
    dfr['Recent_rating_mean'].iloc[i] = reduce(lambda x, y: float(x)+float(y),
                                            dfr.Recent_ratings.iloc[i])/float(len(dfr.Recent_ratings.iloc[i]))
    dfr['Recent_rating_SD'].iloc[i] = np.std(dfr.Recent_ratings.iloc[i], axis=0)

In [9]:
dfr.Total_reviews = dfr.Total_reviews.str.replace(",","")

In [10]:
dfr.Total_reviews=pd.to_numeric(dfr.Total_reviews)

In [11]:
dfr.drop(dfr.Reviewer_ranking=="", axis=0, inplace=True)

In [12]:
dfr.Reviewer_ranking.fillna(20000000, inplace=True)

In [13]:
#dfr[dfr.Reviewer_ranking=='(Learn']

In [14]:
dfr.Reviewer_ranking = dfr.Reviewer_ranking.str.replace(",","")

In [15]:
dfr.drop(dfr[dfr.Reviewer_ranking=="(Learn"].index, axis=0, inplace=True)

In [16]:
#dfr[dfr.Reviewer_ranking=='(Learn']

In [17]:
dfr.Reviewer_ranking=pd.to_numeric(dfr.Reviewer_ranking)

In [18]:
dfr.Total_reviews=pd.to_numeric(dfr.Total_reviews)

In [19]:
dfr.Total_helpful_votes.fillna(0, inplace=True)

In [20]:
dfr.Total_helpful_votes=dfr.Total_helpful_votes.str.split()

In [21]:
dfr.Total_helpful_votes=dfr.Total_helpful_votes

In [22]:
dfr['Total_helpful_votes_nmrtr']=None

In [23]:
dfr['Total_helpful_votes_dnmtr']=0

In [24]:
dfrt=dfr.iloc[0:100]

In [25]:
df = pd.read_csv('Net_Ratings_Robust_backup.csv')

/Users/gregmurray/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
dfr.Total_helpful_votes.fillna(0, inplace=True)

In [27]:
dfr.Total_helpful_votes_nmrtr

1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
         ... 
39248    None
39249    None
39250    None
39251    None
39252    None
39253    None
39254    None
39255    None
39256    None
39257    None
39258    None
39259    None
39260    None
39261    None
39262    None
39263    None
39264    None
39265    None
39266    None
39267    None
39268    None
39269    None
39270    None
39271    None
39272    None
39273    None
39274    None
39275    None
39276    None
39277    None
Name: Total_helpful_votes_nmrtr, Length: 39118, dtype: object

In [28]:
dfr.Total_helpful_votes_nmrtr = dfr.Total_helpful_votes.apply(lambda d: d if d==0 else d[0])

In [29]:
dfr.Total_helpful_votes_dnmtr = dfr.Total_helpful_votes.apply(lambda d: d if d==0 else d[-1])

In [30]:
'''for i in range(8000, 16000):
    if dfr.Total_helpful_votes.iloc[i]==0:
        #print "first if " + str(dfrt.Total_helpful_votes.iloc[i])
        dfr.Total_helpful_votes_nmrtr.iloc[i] = dfr.Total_helpful_votes.iloc[i]
        dfr.Total_helpful_votes_dnmtr.iloc[i] = dfr.Total_helpful_votes.iloc[i]
        continue
    else:
        dfr.Total_helpful_votes_nmrtr.iloc[i] = dfr.Total_helpful_votes.iloc[i][0]
        dfr.Total_helpful_votes_dnmtr.iloc[i] = dfr.Total_helpful_votes.iloc[i][-1]
'''       

'for i in range(8000, 16000):\n    if dfr.Total_helpful_votes.iloc[i]==0:\n        #print "first if " + str(dfrt.Total_helpful_votes.iloc[i])\n        dfr.Total_helpful_votes_nmrtr.iloc[i] = dfr.Total_helpful_votes.iloc[i]\n        dfr.Total_helpful_votes_dnmtr.iloc[i] = dfr.Total_helpful_votes.iloc[i]\n        continue\n    else:\n        dfr.Total_helpful_votes_nmrtr.iloc[i] = dfr.Total_helpful_votes.iloc[i][0]\n        dfr.Total_helpful_votes_dnmtr.iloc[i] = dfr.Total_helpful_votes.iloc[i][-1]\n'

In [31]:
dfp = pd.read_csv('ProductReviews.csv')

In [32]:
rd = dfp.iloc[:, :9]

In [33]:
rd = rd.groupby(['Item_ID', 'Review_date']).agg({'Rating':'sum', 'Review_ID':'count'})

In [34]:
rd.reset_index(inplace=True)

In [35]:
rd['Review_date']=pd.to_datetime(rd.Review_date)

In [36]:
rd = rd.sort_values(by=['Item_ID', 'Review_date'])

In [37]:
rd[(rd.Rating/rd.Review_ID)>5]

,Item_ID,Review_date,Rating,Review_ID


In [38]:
rd.reset_index(drop=True, inplace=True)

In [39]:
#rd

In [40]:
def rol_mean(df):
    try:
        df.sort_values(by=['Item_ID', 'Review_date'])
    except e as IndexError:
        print "Data Frame must have columns 'Item_ID and 'Review_date'"
    res = []
    rol_rate = 0
    rol_revcnt = 0
    rol_mean = 0
    crnt_item = df.iloc[0, 0]
    total_item_revs = 1
    
    def append_tmp(df, tmp, res, rol_mean, total_item_revs):
            tmp.append(df.iloc[i, 0])
            tmp.append(df.iloc[i, 1])
            tmp.append(rol_mean)
            tmp.append(total_item_revs)
            if first_rev:
                tmp.append(True)
            else:
                tmp.append(False)
            res.append(tmp)
            total_item_revs +=1
            return res
            
    for i in range(1,len(df)):
        tmp=[]
        new_item=False
        first_rev=False
        if df.iloc[i, 0]!=crnt_item:
            crnt_item=df.iloc[i, 0]
            rol_rate = 0
            rol_revcnt = 0
            rol_mean = 0
            total_item_revs = 0 
            first_rev = True
            tmp.append(df.iloc[i, 0])
            tmp.append(df.iloc[i, 1])
            tmp.append(rol_mean)
            tmp.append(total_item_revs)
            if first_rev:
                tmp.append(True)
            else:
                tmp.append(False)
            res.append(tmp)
            total_item_revs +=1
            continue
        try:
            if df.iloc[i+1, 0]!=crnt_item:
                new_item=True
        except:
            #print "hmmm"
            pass
        rol_rate = rol_rate + df.iloc[i-1, 2]
        #print df.iloc[i,2]
        #print "i=" + str(i) + " rol_rate: " + str(rol_rate)
        rol_revcnt = rol_revcnt + df.iloc[i-1, 3]
        #print "i=" + str(i) + " rol_revcnt: " + str(rol_revcnt)
        rol_mean = float(rol_rate)/float(rol_revcnt)
        tmp.append(df.iloc[i, 0])
        tmp.append(df.iloc[i, 1])
        tmp.append(rol_mean)
        tmp.append(total_item_revs)
        if first_rev:
            tmp.append(True)
        else:
            tmp.append(False)
        res.append(tmp)
        total_item_revs +=1
            
    return res

In [41]:
#t = rd[(rd['Item_ID']=='B008CSO96I')]

In [42]:
#rol_mean(t)

In [43]:
rol = rol_mean(rd)

In [44]:
#rol

In [45]:
r = pd.DataFrame(rol, columns=['Item_ID', 'Review_date', 'Rol_Mean', 'Total_Item_Reviews', 'First_Review_date'])

In [46]:
rd2 = rd.merge(r, how='left', on=['Item_ID', 'Review_date'])

In [47]:
#dfp[(dfp['Item_ID']=='B008CSO96I')]

In [48]:
#rd2

In [49]:
dfp = dfp.iloc[:, :9]

In [50]:
dfp['Review_date']=pd.to_datetime(dfp.Review_date)

In [51]:
rd2.columns = ['Item_ID', 'Review_date', 'Ratings_Sum', 'Review_Cnt', 'Net_Rating', 'Total_Item_Reviews', 'First_Review_date']

In [52]:
dfp2 = dfp.merge(rd2, how='left', on=['Item_ID', 'Review_date'])

In [53]:
dfp2.sort_values(by=['Item_ID', 'Review_date'], inplace=True)

In [54]:
#dfp2.to_csv('/Users/gregmurray/Documents/Marketing_Analytics_Winter_18/TabletData_Project/Net_Ratings.csv')

In [55]:
#dfp2['Net_Rating_Dsc'] = dfp2['Net_Rating'].round(0)

In [56]:
dfp5 = dfp2.merge(dfr, how='left', on='Reviewer_ID')

In [57]:
#dfp5[dfp5['Item_ID']=='B008CSO96I']['Review_date']

In [58]:
#dfp5[dfp5['Review_ID']=='R2L4YGFE4855W2']['Net_Rating']


In [59]:
#type(dfp2[dfp2['Review_ID']=='RPKL7UKFYY0AL'].Review_date.iloc[0])

In [60]:
#dfp5

In [61]:
dfp5.at[0, 'Net_Rating']=0

In [62]:
dfp5.at[0, 'Review_ID']=0

In [63]:
dfp5.at[0,'First_Review_date']=True

In [64]:
dfp5.First_Review_date.nunique()

2

In [65]:
dfp5.Total_Item_Reviews.fillna(0, inplace=True)

In [66]:
dfp5.Total_reviews.fillna(0, inplace=True)

In [67]:
dfp5.Reviewer_ranking.fillna(20000000, inplace=True)

In [68]:
dfp5.Recent_rating_mean.fillna(0, inplace=True)

In [69]:
dfp5.Recent_rating_SD.fillna(0, inplace=True)

In [70]:
dfp5.Total_helpful_votes_nmrtr.fillna(0, inplace=True)

In [71]:
dfp5.Total_helpful_votes_dnmtr.fillna(0, inplace=True)

In [72]:
dfp5.Recent_ratings.fillna(0, inplace=True)

In [73]:
dfp5[dfp5.Recent_ratings.isnull()==True]

,Review_ID,Item_ID,Base_item_ID,Review_date,Reviewer_ID,Real_name,Verified_purchase,Rating,Title,Ratings_Sum,...,First_Review_date,Total_reviews,Reviewer_ranking,Total_helpful_votes,Location,Recent_ratings,Recent_rating_mean,Recent_rating_SD,Total_helpful_votes_nmrtr,Total_helpful_votes_dnmtr


In [74]:
dfp5.rename(columns={'Total_reviews':'Total_rvwr_reviews'}, inplace=True)

In [75]:
#dfp5.to_csv('/Users/gregmurray/Documents/Marketing_Analytics_Winter_18/TabletData_Project/Net_Ratings_Robust.csv')

In [76]:
dfp5[dfp5.First_Review_date==True][['Net_Rating', 'Rating', 'Review_Cnt', 'First_Review_date']]

,Net_Rating,Rating,Review_Cnt,First_Review_date
0,0.0,5,1,True
23,0.0,1,1,True
24,0.0,5,1,True
28,0.0,5,1,True
29,0.0,3,1,True
31,0.0,4,1,True
37,0.0,4,1,True
39,0.0,4,1,True
40,0.0,4,1,True
41,0.0,1,1,True


In [77]:
dfp5.columns

Index([u'Review_ID', u'Item_ID', u'Base_item_ID', u'Review_date',
       u'Reviewer_ID', u'Real_name', u'Verified_purchase', u'Rating', u'Title',
       u'Ratings_Sum', u'Review_Cnt', u'Net_Rating', u'Total_Item_Reviews',
       u'First_Review_date', u'Total_rvwr_reviews', u'Reviewer_ranking',
       u'Total_helpful_votes', u'Location', u'Recent_ratings',
       u'Recent_rating_mean', u'Recent_rating_SD',
       u'Total_helpful_votes_nmrtr', u'Total_helpful_votes_dnmtr'],
      dtype='object')

In [78]:
'''dfp5["Net_Rating_Bucket"] = dfp5.Net_Rating.apply(lambda x: "A0" if x<1 else ("B1.0" if x<1.25 else (
        "C1.5" if x<1.75 else ("D2.0" if x<2.25 else("E2.5" if x<2.75 else ("F3.0" if x<3.25 else ("G3.5" if x<3.75 else (
                                "H4.0" if x<4.25 else ("I4.5" if x<4.75 else "J5.0")))))))))
                                '''

'dfp5["Net_Rating_Bucket"] = dfp5.Net_Rating.apply(lambda x: "A0" if x<1 else ("B1.0" if x<1.25 else (\n        "C1.5" if x<1.75 else ("D2.0" if x<2.25 else("E2.5" if x<2.75 else ("F3.0" if x<3.25 else ("G3.5" if x<3.75 else (\n                                "H4.0" if x<4.25 else ("I4.5" if x<4.75 else "J5.0")))))))))\n                                '

In [79]:
dfp5["Net_Rating_Bucket"] = dfp5.Net_Rating.apply(lambda x: "A0" if x<1 else ("B1.0" if x<1.5 else (
        "C2.0" if x<2.5 else ("D3.0" if x<3.5 else("E4.0" if x<4.5 else "F5.0")))))

In [80]:
dfp5["Net_Rating_Level"] = dfp5.Net_Rating.apply(lambda x: "negative" if int(x)<3 else ("neutral" if int(x)==3 else (
        "favorable" )))

In [81]:
dfp6=dfp5.copy()

In [82]:
dfp6['Sympathetic_Rating'] = abs(dfp6['Rating']-dfp6['Net_Rating'])<=.5

In [83]:
dfp6

,Review_ID,Item_ID,Base_item_ID,Review_date,Reviewer_ID,Real_name,Verified_purchase,Rating,Title,Ratings_Sum,...,Total_helpful_votes,Location,Recent_ratings,Recent_rating_mean,Recent_rating_SD,Total_helpful_votes_nmrtr,Total_helpful_votes_dnmtr,Net_Rating_Bucket,Net_Rating_Level,Sympathetic_Rating
0,0,1400501776,NaN,2012-04-11,A3LDPF5FMB782Z,True,False,5,Great Color E-Reader With Extra Features,5,...,"[1729, of, 1804]","Houston, Texas, United States","[4, 4, 4, 2, 5, 5, 1, 5, 4, 4]",3.800000,1.249000,1729,1804,A0,negative,False
1,R2UNAEP2B0L62Z,1400501776,NaN,2012-04-17,A1MZVETF38AMZ6,False,False,3,Good Hardware - terrible OS and App availability,3,...,"[274, of, 285]","Earthbound, currently","[5, 3, 5, 5, 5, 5, 5, 4, 5, 3]",4.500000,0.806226,274,285,F5.0,favorable,False
2,R142SLR69LVLT1,1400501776,NaN,2012-04-23,A2FHM5FB0BXKGA,True,False,5,Biggest Deal Around,5,...,"[447, of, 658]",Eastern New England,"[5, 5, 2, 5, 5, 5, 5, 5, 5, 5]",4.700000,0.900000,447,658,E4.0,favorable,False
3,R213BD15OP9Z6N,1400501776,NaN,2012-04-26,A1I3MJJSS3H9RR,False,True,5,"If you can root it, it is better than Amazon's...",9,...,"[6, of, 7]",NaN,"[5, 4]",4.500000,0.500000,6,7,E4.0,favorable,False
4,RJWG7XF0MDVTE,1400501776,NaN,2012-04-26,A2WWBWN4GTCW7Y,False,True,4,Nook tablets,9,...,"[1, of, 1]",NaN,[4],4.000000,0.000000,1,1,E4.0,favorable,True
5,R204PVMFUOSQ2L,1400501776,NaN,2012-05-02,A2USJ3U5OP34N,False,True,5,Nook color tablet 8 gb,5,...,0,NaN,"[5, 5, 4, 5, 3, 5]",4.500000,0.763763,0,0,E4.0,favorable,False
6,R272PFDT40WUP5,1400501776,NaN,2012-05-06,A2WEPBPVYE04UM,False,False,5,Best Gadget Ever!,5,...,"[7, of, 7]",NaN,"[5, 5, 5, 5]",5.000000,0.000000,7,7,F5.0,favorable,True
7,R3DNNXUIXZ4E5Z,1400501776,NaN,2012-05-15,A2DWX3LCX7USSE,False,False,5,Not Complaining,5,...,"[27, of, 29]",NaN,"[4, 5, 4, 5]",4.500000,0.500000,27,29,F5.0,favorable,True
8,RTTDIEG8TE2WF,1400501776,NaN,2012-05-18,A1V1JXSVFAEFBK,False,False,5,Love My Nook Tablet 8GB,5,...,"[2, of, 4]",TEXAS,"[5, 5, 5, 5, 5]",5.000000,0.000000,2,4,F5.0,favorable,True
9,RH662NURKMM2H,1400501776,NaN,2012-05-25,A3AQ5F7I4JGPW6,False,True,5,My New Nook Tablet!,5,...,0,NaN,"[4, 2, 5, 5, 5, 5, 5, 5, 5, 4]",4.500000,0.921954,0,0,F5.0,favorable,True


In [84]:
dfp6.groupby('Sympathetic_Rating').count()

,Review_ID,Item_ID,Base_item_ID,Review_date,Reviewer_ID,Real_name,Verified_purchase,Rating,Title,Ratings_Sum,...,Reviewer_ranking,Total_helpful_votes,Location,Recent_ratings,Recent_rating_mean,Recent_rating_SD,Total_helpful_votes_nmrtr,Total_helpful_votes_dnmtr,Net_Rating_Bucket,Net_Rating_Level
Sympathetic_Rating,,,,,,,,,,,,,,,,,,,,,
False,32379,32379,6382,32379,32368,32379,32379,32379,32379,32379,...,32379,32241,13024,32379,32379,32379,32379,32379,32379,32379
True,8362,8362,1820,8362,8361,8362,8362,8362,8362,8362,...,8362,8318,3599,8362,8362,8362,8362,8362,8362,8362


In [85]:
dfp6["Net_Rating_Class"] = dfp6.Net_Rating.apply(lambda x: "low" if x<3 else ("medium" if x<4 else (
        "high")))

In [86]:
dfp6.to_csv('/Users/gregmurray/Documents/Marketing_Analytics_Winter_18/TabletData_Project/Net_Ratings_Symp.csv')

In [105]:
dfp6.groupby('Item_ID').Total_Item_Reviews.count().sort_values(ascending=False)

Item_ID
B0051VVOB2    19040
B004U78J1G      690
B0045FM6SU      659
B0055D66V4      593
B004EPV7TK      559
B004XZL980      488
B0052P6DQI      436
B004UL34EY      412
B004PGMFG2      391
B004QRIUOG      387
B005HUH88K      377
B002C7481G      357
B0055D67HW      355
B00519RW1U      351
B0047Q9GT4      344
B007P4VOWC      332
B0067PLM5E      319
B007P4YAPK      312
B00413PEZS      298
B005B9G79I      261
B0013FRNKG      251
B004U9USEA      239
B00659AS70      234
B004U78JT8      231
B00519RWI8      228
B004RCUUT8      216
B004SBI2PW      201
B003COZM2C      199
B0047DVWLW      196
B005N130Y4      196
              ...  
B006RW5YCY        1
B006TW5YFY        1
B006UGNYNI        1
B006VUSTVU        1
B0051H7FJG        1
B006VV0F5W        1
B006W49NRY        1
B00519AFOQ        1
B006WX06EE        1
B0050JSKR6        1
B00500598I        1
B004ZMG7BA        1
B004X8PA7S        1
B006ZAEV7M        1
B004TH699Q        1
B006ZBP3ZK        1
B004V5WGHU        1
B006ZT4EPM        1
B0070287TW  

In [88]:
dfv = pd.read_csv('multiTimeline_GT.csv')

In [89]:
dfv.reset_index(drop=True,inplace=True)

In [90]:
dfv.Month = dfv.Month.str.split("-")

In [91]:
dfv['Review_year'] = dfv.Month.apply(lambda d: d[0])

In [92]:
dfv['Review_month'] = dfv.Month.apply(lambda d: d[1])

In [93]:
dfv

,Month,Tablet computer: (United States),Review_year,Review_month
0,"[2004, 01]",10,2004,01
1,"[2004, 02]",10,2004,02
2,"[2004, 03]",10,2004,03
3,"[2004, 04]",9,2004,04
4,"[2004, 05]",9,2004,05
5,"[2004, 06]",9,2004,06
6,"[2004, 07]",9,2004,07
7,"[2004, 08]",10,2004,08
8,"[2004, 09]",10,2004,09
9,"[2004, 10]",9,2004,10


In [94]:
dfm = pd.read_csv('Prod_MrktDyn_Clean.csv')

In [95]:
dfp5

,Review_ID,Item_ID,Base_item_ID,Review_date,Reviewer_ID,Real_name,Verified_purchase,Rating,Title,Ratings_Sum,...,Reviewer_ranking,Total_helpful_votes,Location,Recent_ratings,Recent_rating_mean,Recent_rating_SD,Total_helpful_votes_nmrtr,Total_helpful_votes_dnmtr,Net_Rating_Bucket,Net_Rating_Level
0,0,1400501776,NaN,2012-04-11,A3LDPF5FMB782Z,True,False,5,Great Color E-Reader With Extra Features,5,...,495.0,"[1729, of, 1804]","Houston, Texas, United States","[4, 4, 4, 2, 5, 5, 1, 5, 4, 4]",3.800000,1.249000,1729,1804,A0,negative
1,R2UNAEP2B0L62Z,1400501776,NaN,2012-04-17,A1MZVETF38AMZ6,False,False,3,Good Hardware - terrible OS and App availability,3,...,47124.0,"[274, of, 285]","Earthbound, currently","[5, 3, 5, 5, 5, 5, 5, 4, 5, 3]",4.500000,0.806226,274,285,F5.0,favorable
2,R142SLR69LVLT1,1400501776,NaN,2012-04-23,A2FHM5FB0BXKGA,True,False,5,Biggest Deal Around,5,...,24705.0,"[447, of, 658]",Eastern New England,"[5, 5, 2, 5, 5, 5, 5, 5, 5, 5]",4.700000,0.900000,447,658,E4.0,favorable
3,R213BD15OP9Z6N,1400501776,NaN,2012-04-26,A1I3MJJSS3H9RR,False,True,5,"If you can root it, it is better than Amazon's...",9,...,1995066.0,"[6, of, 7]",NaN,"[5, 4]",4.500000,0.500000,6,7,E4.0,favorable
4,RJWG7XF0MDVTE,1400501776,NaN,2012-04-26,A2WWBWN4GTCW7Y,False,True,4,Nook tablets,9,...,7486709.0,"[1, of, 1]",NaN,[4],4.000000,0.000000,1,1,E4.0,favorable
5,R204PVMFUOSQ2L,1400501776,NaN,2012-05-02,A2USJ3U5OP34N,False,True,5,Nook color tablet 8 gb,5,...,2062801.0,0,NaN,"[5, 5, 4, 5, 3, 5]",4.500000,0.763763,0,0,E4.0,favorable
6,R272PFDT40WUP5,1400501776,NaN,2012-05-06,A2WEPBPVYE04UM,False,False,5,Best Gadget Ever!,5,...,842702.0,"[7, of, 7]",NaN,"[5, 5, 5, 5]",5.000000,0.000000,7,7,F5.0,favorable
7,R3DNNXUIXZ4E5Z,1400501776,NaN,2012-05-15,A2DWX3LCX7USSE,False,False,5,Not Complaining,5,...,884457.0,"[27, of, 29]",NaN,"[4, 5, 4, 5]",4.500000,0.500000,27,29,F5.0,favorable
8,RTTDIEG8TE2WF,1400501776,NaN,2012-05-18,A1V1JXSVFAEFBK,False,False,5,Love My Nook Tablet 8GB,5,...,1968270.0,"[2, of, 4]",TEXAS,"[5, 5, 5, 5, 5]",5.000000,0.000000,2,4,F5.0,favorable
9,RH662NURKMM2H,1400501776,NaN,2012-05-25,A3AQ5F7I4JGPW6,False,True,5,My New Nook Tablet!,5,...,1552331.0,0,NaN,"[4, 2, 5, 5, 5, 5, 5, 5, 5, 4]",4.500000,0.921954,0,0,F5.0,favorable


In [96]:
dfm

,Unnamed: 0,Item_ID,Title,Upc,Week_Num,Sales_Rank,List_Price,Price,LowestNewPrice,LowestUsedPrice,LowestRefurbishedPrice,TradeInValue,TotalNew,TotalUsed,TotalRefurb,NumReviews
0,0,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,1,1.0,0.0,199.00,199.00,185.00,179.00,0.0,1.0,19.0,1.0,13189.0
1,1,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,2,1.0,0.0,199.00,199.00,190.00,179.00,0.0,1.0,15.0,1.0,13622.0
2,2,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,3,1.0,0.0,199.00,199.00,169.50,179.00,0.0,1.0,21.0,1.0,14057.0
3,3,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,4,1.0,0.0,199.00,199.00,175.99,179.00,0.0,1.0,18.0,1.0,14408.0
4,4,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,5,1.0,0.0,199.00,199.00,194.99,169.00,0.0,1.0,19.0,1.0,14739.0
5,5,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,6,1.0,0.0,199.00,199.00,170.00,169.00,0.0,1.0,21.0,1.0,15010.0
6,6,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,7,1.0,0.0,199.00,199.00,165.00,169.00,0.0,1.0,29.0,1.0,15343.0
7,7,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,8,1.0,0.0,199.00,199.00,152.00,169.00,0.0,1.0,36.0,1.0,15672.0
8,8,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,9,1.0,0.0,199.00,199.00,152.00,169.00,0.0,1.0,28.0,1.0,15917.0
9,9,B0051VVOB2,"Kindle Fire, Full Color 7"" Multi-touch Display...",8.150000e+11,10,1.0,0.0,199.00,199.00,156.91,169.00,0.0,1.0,33.0,1.0,16159.0


In [97]:
len(dfp5.Review_date.unique())

1037

In [98]:
dfp5 = dfp5[dfp5.Review_date>='2012-2-1']

In [99]:
dfp7 = dfp7.iloc[0:100]

NameError: name 'dfp7' is not defined

In [ ]:
dfp7

In [ ]:
from datetime import timedelta
import datetime
def add_week_num2(ser):
    start_date = datetime.datetime(2012, 2, 1)
    for i in range(0, 24):
        a_date=start_date + timedelta(days=7*i)
        b_date = a_date+timedelta(days=6)
        if ser>=a_date and ser<=b_date:
            return int(i+1)      
    return None    

In [ ]:
dfp5['Week_Num'] = dfp5.Review_date.apply(add_week_num2)

In [ ]:
dfp7[dfp7.Week_Num==1]["Review_date"]

In [ ]:
dfp5['Review_month'] = dfp5['Review_date'].apply(lambda d: d.month)

In [ ]:
dfp5['Review_year'] = dfp5['Review_date'].apply(lambda d: d.year)

In [ ]:
type(dfp5['Review_month'].iloc[0])

In [ ]:
dfv['Review_month']=dfv['Review_month'].apply(int)

In [ ]:
dfv['Review_year']=dfv['Review_year'].apply(int)

In [ ]:
type(dfv['m'].iloc[0])

In [ ]:
dfp5 = dfp5.merge(dfv, how = 'left', on=['Review_year', 'Review_month'])

In [ ]:
dfm = pd.read_csv('Prod_MrktDyn_Clean.csv')

In [ ]:
dfp5 = dfp5.merge(dfm, how='left' , on=['Item_ID', 'Week_Num'])

In [ ]:
#dfp5.to_csv('/Users/gregmurray/Documents/Marketing_Analytics_Winter_18/TabletData_Project/Net_Ratings_Robust.csv')

In [ ]:
dfp5 = dfp5[dfp5.Price!=0]

In [ ]:
dfpg